In [2]:
import torch
import matplotlib
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import confusion_matrix, accuracy_score
from torch import nn

In [3]:
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, ), (0.5, ))])
test = datasets.FashionMNIST('.', train= False, transform= transforms, download= True)
test_loader = DataLoader(test, batch_size= 64, shuffle= True)

100.0%


Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw



100.0%


Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw



100.0%


Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw



100.0%

Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw



In [4]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Conv2d(1, 32, kernel_size=3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2, 2), stride=2),
                                 nn.Conv2d(32, 64, kernel_size=3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2, 2), stride=2),
                                 nn.Conv2d(64, 32, kernel_size= 3),
                                 nn.ReLU(),
                                 nn.MaxPool2d((2, 2), stride=2))
        self.classify_head = nn.Sequential(nn.Linear(32, 20, bias=True),
                                           nn.Linear(20, 10, bias= True))
    def forward(self, x):
        return self.classify_head(self.net(x).reshape(-1, 32))

In [ ]:
model = CNN()

model = torch.load('./ModelFiles/model.pt', map_location='cuda')  #

for x in model.state_dict().keys():
    print(f'{x} {model.state_dict()[x].size()}')

FileNotFoundError: [Errno 2] No such file or directory: './ModelFiles/model.pt'

In [7]:
all_preds, all_labels = [], []
model.eval()
with torch.no_grad():
    for input, target in test_loader:
        input, target = input.to('cuda'), target.to('cuda')
        output = model(input)
        val, index = torch.max(output, dim= 1)
        all_preds.extend(index.to('cpu'))
        all_labels.extend(target.to('cpu'))
print(accuracy_score(all_preds, all_labels))

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same